# VIDEO CORE ANALYSIS SYSTEM

## Setup: Clone Repository


In [ ]:
!git clone https://github.com/fuadnafiz98/video-core.git
%cd video-core

### Install Dependencies

In [ ]:
!pip install scenedetect[opencv] click questionary numpy -q

### Upload Video

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

print("Upload your video file:")
uploaded = files.upload()

video_dir = Path("videos")
video_dir.mkdir(exist_ok=True)

for filename in uploaded.keys():
    shutil.move(filename, video_dir / filename)
    print(f"Moved {filename} to videos/")

### Run Analysis

In [ ]:
from src.extractors.shot_cut_detector import ShotCutDetector
from src.extractors.motion_analyzer import MotionAnalyzer
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import json
import time

videos = [f for f in Path("videos").glob("*.*") if f.suffix.lower() in ['.mp4', '.avi', '.mov', '.mkv', '.webm']]

if not videos:
    print("No videos found. Please upload a video first.")
else:
    video_path = videos[0]
    print(f"\nProcessing: {video_path.name}\n")

    start_time = time.time()

    detector = ShotCutDetector(threshold=27.0, min_scene_len=15)
    analyzer = MotionAnalyzer(sample_rate=5, downscale=2)

    print("Running shot detection and motion analysis in parallel...\n")

    with ThreadPoolExecutor(max_workers=2) as executor:
        shot_future = executor.submit(detector.extract, str(video_path))
        motion_future = executor.submit(analyzer.extract, str(video_path))

        shot_result = shot_future.result()
        motion_result = motion_future.result()

    elapsed_time = time.time() - start_time

    output_data = {
        "video_file": str(video_path),
        "processing_time_seconds": round(elapsed_time, 2),
        "features": {
            "shot_cuts": shot_result,
            "motion": motion_result
        }
    }

    results_dir = Path("results") / video_path.stem
    results_dir.mkdir(parents=True, exist_ok=True)
    output_file = results_dir / "output.json"

    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=2)

    print("\n" + "=" * 50)
    print("ANALYSIS COMPLETE")
    print("=" * 50)
    print(f"  PROCESSING TIME: {elapsed_time:.2f}s")
    print(f"  CUTS DETECTED: {shot_result['total_cuts']}")
    print(f"  AVG SCENE: {shot_result['avg_scene_length']}s")
    print(f"  DURATION: {shot_result['duration']}s")
    print(f"  MOTION: P90={motion_result['p90_motion']:.2f} ({motion_result['motion_intensity']}) | AVG={motion_result['average_motion']:.2f} | MAX={motion_result['max_motion']:.2f}")
    print(f"  SAMPLES: {motion_result['sampled_frames']}")
    print(f"\nResults saved to: {output_file}")
    print("=" * 50)

### View Results

In [ ]:
import json
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

if result_files:
    with open(result_files[0]) as f:
        data = json.load(f)
    print(json.dumps(data, indent=2))
else:
    print("No results found.")

### Download Results

In [ ]:
from google.colab import files
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

for result_file in result_files:
    files.download(str(result_file))